# Create a metadata df of unique drugs from a set of docking results

The metadata contains `Inchi-key`, `TITLE`, `SMILES`.<br>
The final file `OZD.May29.unique.csv` is passed to Kyle to compute molecular features (primarily Mordred descriptors) on HPC.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import sys
from pathlib import Path
from time import time
import numpy as np
import pandas as pd
from glob import glob
import matplotlib
import matplotlib.pyplot as plt
print( Path.cwd() )

/vol/ml/apartin/projects/covid-19/mol-features/nbs


In [11]:
datadir = Path('../data/raw/OZD-dock-2020-06-01/OZD')
outdir = datadir.parent
files = sorted(datadir.glob('*4col.csv'))
len(files)

35

In [12]:
# df1=pd.read_csv(files[0]) # .sort_values('TITLE')
# df2=pd.read_csv(files[1]) # .sort_values('TITLE')
# print(df1.shape)
# print(df2.shape)

# df = pd.concat([df1, df2], axis=0)
# df = df.drop_duplicates(subset=['TITLE'])
# print(df.shape)

In [13]:
df = None
t0 = time()

for i, f in enumerate(files):
    print(f'Loading {i+1} ... {f.name}')
    df_in = pd.read_csv(f)
    
    cols = ['Inchi-key','SMILES','TITLE']
    if all([True if c in df_in.columns else False for c in cols]):
        df_in = df_in[cols]
    else:
        print(f'    skip {f}')
        continue
        
    if df is None:
        df = df_in.reset_index(drop=True)
    else:
        df = pd.concat([df, df_in], axis=0, ignore_index=True)
        df = df.drop_duplicates(subset=['TITLE'])
        df = df.reset_index(drop=True)

runtime = time() - t0
print('\nRuntime: {:.2f} mins'.format( runtime/60 ))


Loading 1 ... 3CLPro_7BQY_A_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 2 ... ADRP_6W02_A_1_H.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 3 ... NPRBD_6VYO_AB_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 4 ... NPRBD_6VYO_A_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 5 ... NPRBD_6VYO_BC_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 6 ... NPRBD_6VYO_CD_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 7 ... NPRBD_6VYO_DA_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 8 ... NSP10-16_6W61_AB_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 9 ... NSP10-16_6W61_AB_2_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 10 ... NSP10_6W61_B_1_F.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 11 ... NSP13_HELICASE_5WWP_0_A_A__DU__apo_PHE_A-145.Orderable_zinc_db_enaHLL.sorted.4col.csv
    skip ../data/raw/docking-2020-06-01/all/OZD/NSP13_HELICASE_5WWP_0_A_A__DU__apo_PHE_A-145.Orderable_zinc_db_enaHLL.sorted.4col.csv
Loading 12 ... NSP13_HELICASE_5W

In [16]:
print(df.shape)
df.to_csv(outdir/'OZD.May29.unique.csv', index=False)

(6678494, 3)
